In [8]:
import pandas as pd
import requests
import praw
from praw.models import MoreComments, comment_forest
import pysentiment2 as ps
import demoji
from collections import OrderedDict
from collections import defaultdict
from prawcore.exceptions import Forbidden
from textblob import TextBlob
import nltk
from twitter_scraper import get_tweets
import tweepy

In [11]:
reddit = praw.Reddit("***")

In [12]:
print(reddit.user.me())

sustainablesentiment


In [6]:
targetsubreddits = ["greeninvestor", "sustainability", "climateoffensive", "esginvesting"]

In [7]:
subcomments = OrderedDict.fromkeys(targetsubreddits,[])

In [8]:
dsubcomments = defaultdict(list)

In [9]:
for keys in subcomments:
    for submission in reddit.subreddit(keys).top("all"):
        for top_level_comment in submission.comments[1:]:
            if isinstance(top_level_comment, MoreComments):
                continue
            dsubcomments[keys].append(top_level_comment.body)
            dsubcomments[keys].append(top_level_comment.created_utc)

In [10]:
dsubcomments

defaultdict(list,
            {'greeninvestor': ['Woah!!! This is huge!',
              1580691698.0,
              'Too bad they can’t terminate from all sectors like what the BBC does.',
              1580702805.0,
              'Just yesterday I came across an article on LinkedIn titled [Repeat after me, ‘Canada is uninhabitable without fossil fuels’](https://albertapressleader.ca/repeat-after-me-canada-is-uninhabitable-without-fossil-fuels/).\n\nI went and looked up the profile of both the person who provided the content in the article along with the person who posted it on LinkedIn. Among them they;\n\n- Own a consulting company in the oil and gas sector\n\n- Wrote and published a book about the importance of said sector\n\n- Sit on the board for a few companies in the oil exploration sector\n\n- Had an extensive history of working for companies in the Alberta oil trade\n\nAlways follow the money. It\'s especially sad when you see people gleefully posing with their "I LOVE OIL AND

In [11]:
subcommentsdf = pd.DataFrame().from_dict(dsubcomments, orient = "index")

In [12]:
subcommentsdf = subcommentsdf.transpose()

In [13]:
columns1 = []

for i in targetsubreddits:
    columns1.append(i)
    if columns1 == targetsubreddits:
        for i in targetsubreddits:
            columns1.append(f"{i}-timestamp")

In [14]:
subcommentsdf = pd.DataFrame(subcommentsdf.values.reshape(-1, len(columns1)), columns = [columns1])

In [15]:
order1 = []

for i in targetsubreddits:
    order1.append(i)
    order1.append(i + "-timestamp")

In [16]:
subcommentsdf = subcommentsdf.reindex(columns= [order1])

In [17]:
subcommentsdf

,greeninvestor,greeninvestor-timestamp,sustainability,sustainability-timestamp,climateoffensive,climateoffensive-timestamp,esginvesting,esginvesting-timestamp
0,Woah!!! This is huge!,1.58069e+09,There's also the option of just ditching the p...,1.60754e+09,"Whether you're an adult or not, you can [lobby...",1.57613e+09,Very nicely done,1.6197e+09
1,Too bad they can’t terminate from all sectors ...,1.5807e+09,Wow this is awesome - I remember writing to Co...,1.60754e+09,[deleted],1.57612e+09,Nice work!\nAre you collecting the ratings man...,1.61974e+09
2,Just yesterday I came across an article on Lin...,1.58074e+09,I love the ring tho,1.60754e+09,It’s 3:48 am and all I can think is I gotta wa...,1.57615e+09,Done; hope to see the results!,1.61767e+09
3,This made me feel good. \n\nI subscribe to the...,1.58074e+09,How easily can it be dislodged? That looks lik...,1.60754e+09,Worth noting that Person of the Year isn't rea...,1.57617e+09,I have made a video on this topic check it out...,1.61485e+09
4,This is huge. I wonder who will follow.,1.58071e+09,[deleted],1.60754e+09,I did not care about her being TIME's person o...,1.57617e+09,Okay I have been researching this a lot lately...,1.61223e+09
...,...,...,...,...,...,...,...,...
1229,None,NaN,Why do you use these towls?,1.61858e+09,None,NaN,None,NaN
1230,None,NaN,Bidets are the best thing to come out of the l...,1.61858e+09,None,NaN,None,NaN
1231,None,NaN,Smart move with keeping toilet paper for guests.,1.61859e+09,None,NaN,None,NaN
1232,None,NaN,"Use water to clean yourself, instead of wastin...",1.6185e+09,None,NaN,None,NaN


In [18]:
body = pd.Series(subcommentsdf[subcommentsdf.columns[::2]].values.ravel())

In [19]:
timestamp = pd.Series(subcommentsdf[subcommentsdf.columns[1::2]].values.ravel())

In [20]:
subcomdf = pd.concat([body.rename("body"), timestamp.rename("timestamp")], axis = 1).dropna()

In [35]:
subcomdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2638 entries, 0 to 4933
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   body       2638 non-null   object
 1   timestamp  2638 non-null   object
dtypes: object(2)
memory usage: 61.8+ KB


In [13]:
targetsearchwords = ["esg", "sustainable investing", "responsible investment", "impact investing", "sustainable finance", "green finance", "corporate social responsibility", "esg investing", "socially responsible investing", "climate funds", "portfolio carbon footprint", "carbon finance", "positive screening", "best in class ESG", "sustainability index", "thematic investing", "triple bottom line"]

In [17]:
searchposts = OrderedDict.fromkeys(targetsearchwords,[])
dsearchwords = defaultdict(list)


In [38]:
for keys in searchposts:
    for sub in reddit.subreddit("all").search(keys):
        dsearchwords[keys].append(sub.title)
        dsearchwords[keys].append(sub.created)

In [39]:
searchwordsdf = pd.DataFrame().from_dict(dsearchwords, orient="index")

In [48]:
searchwordsdf

,esg,esg-timestamp,sustainable investing,sustainable investing-timestamp,responsible investment,responsible investment-timestamp,impact investing,impact investing-timestamp,sustainable finance,sustainable finance-timestamp,...,positive screening,positive screening-timestamp,best in class ESG,best in class ESG-timestamp,sustainability index,sustainability index-timestamp,thematic investing,thematic investing-timestamp,triple bottom line,triple bottom line-timestamp
0,A crackdown on miners in China would radically...,1.62165e+09,Blackrock hired me to make sustainable investi...,1.61697e+09,Shibacoin friends! I found a social network th...,1.61948e+09,"Hi all, are there any cardano devs around for ...",1.62131e+09,Bank of America to deploy $1 trillion for sust...,1.618e+09,...,Positive screening for trisomy 18 in first tri...,1.61895e+09,PRPB - CC Capital/Chinh Chu's next SPAC - DD,1.61006e+09,Business of Fashion's Sustainability Index (A....,1.61646e+09,"116 days til the Exam: Dear diary, today I lea...",1.61204e+09,Free Webinar: An introductory course on the pr...,1.61896e+09
1,Shark Tank’s Kevin O'Leary Expects Flood of In...,1.62202e+09,Anyone on this sub interested in Sustainable I...,1.61591e+09,[Business] - Chapel Hill Denham joins UN-suppo...,1.62196e+09,Elon/Devs - thanks for all your efforts. My su...,1.622e+09,My master thesis in sustainable finance requir...,1.62204e+09,...,TIC: positive screening for Down syndrome,1.58e+09,Socially responsible funds that promise to ali...,1.54516e+09,Fashion's biggest firms are making slow progre...,1.61652e+09,Thematic Investing on the Rise: ARK ETFs Leadi...,1.61675e+09,Free Webinar: An introductory course on the pr...,1.61896e+09
2,As somebody looking to be passively invested a...,1.61876e+09,Anyone on this sub interested in Sustainable I...,1.61591e+09,[Business] - Chapel Hill Denham joins UN-suppo...,1.62196e+09,Why impact investing needs a standardised clim...,1.6213e+09,What development would most effectively increa...,1.61884e+09,...,Update: Positive screening for DS,1.58202e+09,I think I solved the Rubix Cube and... it is s...,1.62047e+09,Fashion's biggest firms are making slow progre...,1.61653e+09,"Puru Saxena on High-Growth, Thematic Investing",1.61781e+09,Free Webinar: An introductory course on the pr...,1.61896e+09
3,ESG Global All Cap UCITS ETF,1.62153e+09,Former BlackRock exec Tariq Fancy on why susta...,1.61913e+09,A responsible investment for residential /Comm...,1.61957e+09,"First Swell, now COIN - are there reliable imp...",1.61754e+09,The German government adopts groundbreaking su...,1.62068e+09,...,Waiting for official diagnosis after two posit...,1.58767e+09,Invest like the Norwegian Oil Fund,1.62166e+09,@Reuters: Campaign group asks S&P to cut Adani...,1.616e+09,Thematic Investing Part 1 - Mental Health,1.6139e+09,Triple Bottom Line: Feminism is still relevant...,1.61739e+09
4,Anyone have a portfolio that is majority ESG E...,1.62064e+09,Given the current trend towards sustainable in...,1.618e+09,[World] - 4 steps to take to achieve socially ...,1.62081e+09,"Impact Investing dream, completely unsure of h...",1.61802e+09,My master thesis in sustainable finance requir...,1.62204e+09,...,Negative then positive screenings,1.58778e+09,Bill Ackman is about to merge Stripe and Plaid...,1.62165e+09,Campaign group asks S&P to cut Adani Ports fro...,1.616e+09,The Rise and Rise of Thematic Investing,1.61395e+09,[Book] Beyond the Triple Bottom Line: Eight St...,1.60097e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Êtes vous intéressée par l'investissement dura...,1.61592e+09,"Brian Deese, the incoming director of the Nati...",1.60765e+09,Responsible Investment & Islamic Finance: 10 y...,1.5036e+09,Do Impact investing firms hire Environmental g...,1.59794e+09,[News] Green City Finance Directory Helps Citi...,1.60431e+09,...,NIPT screening results today,1.58986e+09,None,NaN,Oxford University and BlackRock unite for foss...,1.

In [41]:
searchwordsdf = searchwordsdf.transpose()

In [43]:
columns2 = []

for i in targetsearchwords:
    columns2.append(i)
    if columns2 == targetsearchwords:
        for i in targetsearchwords:
            columns2.append(f"{i}-timestamp")

In [44]:
searchwordsdf = pd.DataFrame(searchwordsdf.values.reshape(-1, len(columns2)), columns = [columns2])

In [46]:
order2 = []

for i in targetsearchwords:
    order2.append(i)
    order2.append(i + "-timestamp")

In [47]:
searchwordsdf = searchwordsdf.reindex(columns= [order2])

In [49]:
body3 = pd.Series(searchwordsdf[searchwordsdf.columns[::2]].values.ravel())

In [50]:
body3

0       A crackdown on miners in China would radically...
1       Blackrock hired me to make sustainable investi...
2       Shibacoin friends! I found a social network th...
3       Hi all, are there any cardano devs around for ...
4       Bank of America to deploy $1 trillion for sust...
                              ...                        
1695              Positive carrier screening- next steps?
1696                                                 None
1697    Bisons are already present in the bielorussian...
1698    Investments and Personal Finance Thread - Febr...
1699    What is the plausibility of a high level organ...
Length: 1700, dtype: object

In [52]:
timestamp3

0       1.62165e+09
1       1.61697e+09
2       1.61948e+09
3       1.62131e+09
4         1.618e+09
           ...     
1695     1.6125e+09
1696            NaN
1697    1.60685e+09
1698    1.58211e+09
1699    1.56579e+09
Length: 1700, dtype: object

In [51]:
timestamp3 = pd.Series(searchwordsdf[searchwordsdf.columns[1::2]].values.ravel())

In [53]:
searchwddf = pd.concat([body3.rename("body"), timestamp3.rename("timestamp")], axis = 1).dropna()

In [55]:
searchwddf

,body,timestamp
0,A crackdown on miners in China would radically...,1.62165e+09
1,Blackrock hired me to make sustainable investi...,1.61697e+09
2,Shibacoin friends! I found a social network th...,1.61948e+09
3,"Hi all, are there any cardano devs around for ...",1.62131e+09
4,Bank of America to deploy $1 trillion for sust...,1.618e+09
...,...,...
1694,[News] - Bank of England criticised for financ...,1.6116e+09
1695,Positive carrier screening- next steps?,1.6125e+09
1697,Bisons are already present in the bielorussian...,1.60685e+09
1698,Investments and Personal Finance Thread - Febr...,1.58211e+09


In [42]:
redditdf = pd.concat((subcomdf, searchwddf), axis = 0)

In [43]:
redditdf

,body,timestamp
0,Woah!!! This is huge!,1.58069e+09
1,There's also the option of just ditching the p...,1.60754e+09
2,"Whether you're an adult or not, you can [lobby...",1.57613e+09
3,Very nicely done,1.6197e+09
4,Too bad they can’t terminate from all sectors ...,1.5807e+09
...,...,...
1694,[Business] - China’s central bank plans to fin...,1.60754e+09
1695,[Volin] Source: Covid will be considered a non...,1.59571e+09
1697,Aquaculture companies,1.55792e+09
1698,Investments and Personal Finance Thread - Marc...,1.58453e+09


In [44]:
redditdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4317 entries, 0 to 1699
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   body       4317 non-null   object
 1   timestamp  4317 non-null   object
dtypes: object(2)
memory usage: 101.2+ KB


In [14]:
searchcomments = OrderedDict.fromkeys(targetsearchwords,[])
dsearchcomments = defaultdict(list)

In [46]:
for keys in searchposts:
    for sub in reddit.subreddit("all").search(keys):
        for top_level_comment in sub.comments[1:]:
            if isinstance(top_level_comment, MoreComments):
                continue
            dsearchcomments[keys].append(top_level_comment.body)
            dsearchcomments[keys].append(top_level_comment.created_utc)

In [47]:
dsearchcomments

defaultdict(list,
            {'esg': ['One thing to note is that there\'s barely any regulation of what constitutes ESG investing. Indeed, the best I\'ve heard is "socially responsible investing is whatever the investor believes it is." For me, the best example of this is Shell. Oil producing companies can\'t be green can they? Well, most esg scores are comparing a company to its overall industry so it may well come out looking very green. They also invest in renewables so depending on the screens used it may look like a green investment. \n\nThere were a few WSJ investigations and found that there were essentially zero correlation between esg scores of one provider compared to another, and that many funds had just changed their name to include the word" green" or "sustainable" etc without changing the portfolio.\n\nTheoretically speaking, imagine a company like PG Tips. How can one accurately or precisely estimate its environmental impact? Are they going to annually sample their farm

In [48]:
searchcommentsdf = pd.DataFrame().from_dict(dsearchcomments, orient="index")
searchcommentsdf = searchcommentsdf.transpose()

In [49]:
searchcommentsdf = pd.DataFrame(searchcommentsdf.values.reshape(-1, len(columns2)), columns = [columns2])

In [50]:
searchcommentsdf = searchcommentsdf.reindex(columns= [order2])

In [52]:
body4 = pd.Series(searchcommentsdf[searchcommentsdf.columns[::2]].values.ravel())

In [53]:
timestamp4 = pd.Series(searchcommentsdf[searchcommentsdf.columns[1::2]].values.ravel())

In [54]:
searchcomdf = pd.concat([body4.rename("body"), timestamp4.rename("timestamp")], axis = 1).dropna()

In [56]:
searchcomdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4823 entries, 0 to 14446
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   body       4823 non-null   object
 1   timestamp  4823 non-null   object
dtypes: object(2)
memory usage: 113.0+ KB


In [57]:
redditdf = pd.concat((subcomdf, searchwddf, searchcomdf), axis = 0)

In [15]:
searchcomments = OrderedDict.fromkeys(targetsearchwords,[])

dsearchcomments = defaultdict(list)

In [18]:
for keys in searchposts:
    for sub in reddit.subreddit("all").search(keys):
        for top_level_comment in sub.comments[1:]:
            if isinstance(top_level_comment, MoreComments):
                continue
            dsearchcomments[keys].append(top_level_comment.body)
            dsearchcomments[keys].append(top_level_comment.created_utc)

In [19]:
dsearchcomments

defaultdict(list,
            {'esg': ['6.25 every fucking block til 2024\n\nfew understand this concept.',
              1621628204.0,
              'Is China mining actually dirtier?  I thought they use a lot of hydro\n\nAgree that it would overall be a net positive though',
              1621618657.0,
              'Something I think about a lot is. What would happen to Bitcoin transactions if all of China miners went off line?\n\nDo transactions fees increase.... Does it become easier for the little guy to use his PC to mine?',
              1621621113.0,
              "you don't have the faintest clue what a china mining ban would do for the market. why wouldn't they just move their operation over the border? they already move regularly to capture hydro in the rainy season. markets are dynamic, and so is bitcoin difficulty and price.",
              1621619382.0,
              '[deleted]',
              1621619600.0,
              "I've got an 8 inch dick! *3 inches cutoff* Oh, my

In [20]:
searchcommentsdf = pd.DataFrame().from_dict(dsearchcomments, orient="index")

In [29]:
searchcommentsdf

,esg,esg-timestamp,sustainable investing,sustainable investing-timestamp,responsible investment,responsible investment-timestamp,impact investing,impact investing-timestamp,sustainable finance,sustainable finance-timestamp,...,positive screening,positive screening-timestamp,best in class ESG,best in class ESG-timestamp,sustainability index,sustainability index-timestamp,thematic investing,thematic investing-timestamp,triple bottom line,triple bottom line-timestamp
0,6.25 every fucking block til 2024\n\nfew under...,1.62163e+09,"As a matter or principal, we really ought to b...",1.61696e+09,[https://discord.gg/9EhXr3vaTy](https://disco...,1.6198e+09,I think you'd get a better response if you pos...,1.6213e+09,Sounds like feel good PR BS.,1.61799e+09,...,My God I'm so sorry. I hope you can get some ...,1.61893e+09,Already IN below $10.\n\nLET'S GOOOOOOOOOOOO ...,1.61004e+09,Thanks for sharing this and getting the discus...,1.61647e+09,But what about memematic investing? Haha. You ...,1.61203e+09,You can't spell sustainability without P-R-O-F...,1.61901e+09
1,Is China mining actually dirtier? I thought t...,1.62162e+09,If the situation in Texas and the handling of ...,1.61696e+09,"Take it easy my friend, try not to spam to boa...",1.61955e+09,What's your timezone?,1.62129e+09,Sustainable finance? End QE and we'll see what...,1.61799e+09,...,I'm so sorry you're going through this. I'd hi...,1.61893e+09,Nice one!,1.61005e+09,Sustainability is a pretty meaningless term th...,1.61646e+09,If anyone wonders\n\n* Thematic - more passive...,1.61205e+09,Remind me!,1.61897e+09
2,Something I think about a lot is. What would h...,1.62162e+09,"In the wake of the carbon pricing ruling, this...",1.61694e+09,RT @Derek_Hanekom: Great news! Good for the We...,1.61901e+09,""" I’m building an enterprise support organisat...",1.62132e+09,Deadass why can the government be the bank? Ha...,1.618e+09,...,Normal ultrasound and normal echo,1.61895e+09,"Off the top of my head here, but I believe Chu...",1.61005e+09,Interesting post! Thanks for sharing 😁I have n...,1.61649e+09,Isnt this level one material?,1.61203e+09,[https://sanet.st/blogs/ebooksa/the\_beyond\_t...,1.60095e+09
3,you don't have the faintest clue what a china ...,1.62162e+09,"Good article. \n\n""Sustainable"" investing has ...",1.61694e+09,"When I was little, I wanted to be a police off...",1.56922e+09,"I recommended to space exploration, ocean and ...",1.62197e+09,Because if Manchin won’t go for filibuster ref...,1.618e+09,...,I'm not usually one to post on a subject like ...,1.61892e+09,In 335 @ 10.56!,1.61107e+09,The Workers Rights section is of particular in...,1.61652e+09,I thought that this was a part of the level 1 ...,1.61203e+09,Is this a fictional business or real one? One ...,1.58979e+09
4,[deleted],1.62162e+09,I hate to spoil it for you guys but if you pre...,1.61696e+09,I was always told that life will get worse and...,1.56923e+09,Love this!,1.62197e+09,Does anyone actually trust these bastards? Giv...,1.61804e+09,...,I’m so sorry that you and your husband are goi...,1.61893e+09,"Already in, thanks for DD",1.61003e+09,Thank you for sharing! I like how they broke i...,1.61647e+09,What is the difference (L1 candidate here),1.61202e+09,"""Nuveen ESG Large Cap ETF, it feels a little d...",1.58448e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,...,None,NaN,You may find these links helpful:\n\n- [401(k)...,1.48938e+09,None,NaN,None,NaN,None,NaN
1086,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,...,None,NaN,Very well thought out post. I enjoyed your use...,1.30223e+09,None,NaN,None,NaN,None,NaN
1087,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,...,None,NaN,"""I am a fiscally conservative, socially progre...",1.30223e+09,None,NaN,None,NaN,None,NaN
1088,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,...,None,NaN,"Great thoughtful post, but so why did you ever...",1.30223e+09,None,NaN,None,NaN,Non

In [22]:
searchcommentsdf = searchcommentsdf.transpose()

In [25]:
searchcommentsdf = pd.DataFrame(searchcommentsdf.values.reshape(-1, len(columns2)), columns = [columns2])

In [28]:
searchcommentsdf = searchcommentsdf.reindex(columns= [order2])

In [30]:
body4 = pd.Series(searchcommentsdf[searchcommentsdf.columns[::2]].values.ravel())

In [31]:
body4

0        6.25 every fucking block til 2024\n\nfew under...
1        As a matter or principal, we really ought to b...
2         [https://discord.gg/9EhXr3vaTy](https://disco...
3        I think you'd get a better response if you pos...
4                             Sounds like feel good PR BS.
                               ...                        
18525                                                 None
18526    Thanks for sharing dude.  It encourages me to ...
18527                                                 None
18528                                                 None
18529                                                 None
Length: 18530, dtype: object

In [32]:
timestamp4 = pd.Series(searchcommentsdf[searchcommentsdf.columns[1::2]].values.ravel())

In [33]:
timestamp4

0        1.62163e+09
1        1.61696e+09
2         1.6198e+09
3         1.6213e+09
4        1.61799e+09
            ...     
18525            NaN
18526    1.30224e+09
18527            NaN
18528            NaN
18529            NaN
Length: 18530, dtype: object

In [34]:
searchcomdf = pd.concat([body4.rename("body"), timestamp4.rename("timestamp")], axis = 1).dropna()

In [37]:
searchcomdf.head(18528)

,body,timestamp
0,6.25 every fucking block til 2024\n\nfew under...,1.62163e+09
1,"As a matter or principal, we really ought to b...",1.61696e+09
2,[https://discord.gg/9EhXr3vaTy](https://disco...,1.6198e+09
3,I think you'd get a better response if you pos...,1.6213e+09
4,Sounds like feel good PR BS.,1.61799e+09
...,...,...
18458,You may find these links helpful:\n\n- [401(k)...,1.48938e+09
18475,Very well thought out post. I enjoyed your use...,1.30223e+09
18492,"""I am a fiscally conservative, socially progre...",1.30223e+09
18509,"Great thoughtful post, but so why did you ever...",1.30223e+09
